<a href="https://colab.research.google.com/github/louelidrissi/LLM_Driver_Assistant/blob/main/Driver_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()
file_name = list(uploaded.keys())[0]  # get the uploaded file name


In [ ]:
import json

data = []
with open(file_name, 'r', encoding='utf-8') as f:
    for line in f:
        record = json.loads(line)
        data.append(record)
print(f"Loaded {len(data)} records")


Loaded 24724 records


In [7]:
uploaded = files.upload()  # upload your testing file
test_filename = list(uploaded.keys())[0]

Saving output.jsonl to output.jsonl


In [ ]:
import json

test_data = []
with open(test_filename, 'r', encoding='utf-8') as f:
    for line in f:
        record = json.loads(line)
        test_data.append(record)
print(f"Loaded {len(test_data)} testing records")


Loaded 4945 testing records


In [ ]:
print(f"Testing file uploaded: {test_filename}")
print(f"Training file uploaded: {file_name}")

Testing file uploaded: testing_input_advice_20.jsonl
Training file uploaded: training_input_advice_80.jsonl


In [3]:
!git clone https://github.com/tloen/alpaca-lora.git
%cd alpaca-lora

Cloning into 'alpaca-lora'...
remote: Enumerating objects: 607, done.
remote: Total 607 (delta 0), reused 0 (delta 0), pack-reused 607 (from 1)
Receiving objects: 100% (607/607), 27.84 MiB | 7.53 MiB/s, done.
Resolving deltas: 100% (357/357), done.
/content/alpaca-lora


In [4]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
!pip install bitsandbytes==0.37.2
!pip install -r requirements.txt


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-a0fqdpph
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-a0fqdpph
  Resolved https://github.com/huggingface/peft.git to commit 2813b9c4bfd240408de398028da722ed52528cdb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.8 MB/s eta 0:00:00
  Created wheel for peft: filename=peft-0.17.2.dev0-py3-none-any.whl size=543744 sha256=18827452a7f31d83e86e66a861a488603471695018b16d1d803c95b04a0647bc
  Store

In [5]:
import importlib.util

def check_package_installed(package_name):
    if importlib.util.find_spec(package_name) is not None:
        print(f"{package_name} is installed.")
    else:
        print(f"{package_name} is NOT installed.")

# Example usage:
packages = ['torch', 'transformers', 'peft', 'bitsandbytes', 'fire']
for pkg in packages:
    check_package_installed(pkg)







torch is installed.
transformers is installed.
peft is installed.
bitsandbytes is installed.
fire is installed.


In [12]:
import torch
from transformers import LlamaForCausalLM
from transformers import LlamaTokenizer

import torch
torch.cuda.empty_cache()


base_model = "baffo32/decapoda-research-llama-7B-hf"

model = LlamaForCausalLM.from_pretrained(base_model,  device_map="auto") # remove load_in_8bit=True,
tokenizer = LlamaTokenizer.from_pretrained(base_model)

torch.save(model.state_dict(), 'model_weights.pth')

output_dir = "./lora-alpaca"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)




The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ImportError: Using `bitsandbytes` 8-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [9]:
!pip install --upgrade bitsandbytes


In [11]:
import bitsandbytes as bnb
print(bnb.__version__)


0.48.1


In [7]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 11.6 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.37.2
    Uninstalling bitsandbytes-0.37.2:
      Successfully uninstalled bitsandbytes-0.37.2


In [10]:
!python finetune.py \
  --base_model 'baffo32/decapoda-research-llama-7B-hf' \
  --data_path './training_input_advice.jsonl' \
  --val_data_path './output.jsonl' \
  --output_dir './lora-alpaca-output' \
  --batch_size 128 \
  --micro_batch_size 4 \
  --num_epochs 3 \
  --learning_rate 2e-4 \
  --cutoff_len 512 \
  --val_set_size 150 \
  --lora_r 16 \
  --lora_alpha 16 \
  --lora_dropout 0.05 \
  --lora_target_modules '[q_proj,k_proj,v_proj,o_proj]' \
  --train_on_inputs True \
  --group_by_length


2025-10-18 19:15:17.595374: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760814917.639833   23990 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760814917.650608   23990 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760814917.689954   23990 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760814917.689991   23990 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760814917.689999   23990 computation_placer.cc:177] computation placer alr

In [ ]:
# Install PyTorch with CUDA 11.7 support - adjust based on your environment
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117

# Hugging Face libraries
!pip install transformers datasets accelerate peft bitsandbytes

# Additional dependencies
!pip install scipy tqdm sentencepiece



Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 7.7 MB/s eta 0:00:00


In [9]:
dataset = "/content/alpaca-lora/training_input_advice.jsonl"
for i, data_point in enumerate(dataset):
    if "advice" not in data_point:
        print(f"Missing 'advice' at line {i}")


Missing 'advice' at line 0
Missing 'advice' at line 1
Missing 'advice' at line 2
Missing 'advice' at line 3
Missing 'advice' at line 4
Missing 'advice' at line 5
Missing 'advice' at line 6
Missing 'advice' at line 7
Missing 'advice' at line 8
Missing 'advice' at line 9
Missing 'advice' at line 10
Missing 'advice' at line 11
Missing 'advice' at line 12
Missing 'advice' at line 13
Missing 'advice' at line 14
Missing 'advice' at line 15
Missing 'advice' at line 16
Missing 'advice' at line 17
Missing 'advice' at line 18
Missing 'advice' at line 19
Missing 'advice' at line 20
Missing 'advice' at line 21
Missing 'advice' at line 22
Missing 'advice' at line 23
Missing 'advice' at line 24
Missing 'advice' at line 25
Missing 'advice' at line 26
Missing 'advice' at line 27
Missing 'advice' at line 28
Missing 'advice' at line 29
Missing 'advice' at line 30
Missing 'advice' at line 31
Missing 'advice' at line 32
Missing 'advice' at line 33
Missing 'advice' at line 34
Missing 'advice' at line 35
Mi

In [11]:
!pip install triton

In [8]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.8 MB/s eta 0:00:00


In [2]:
!pip install wandb
import wandb
wandb.login()


wandb: Currently logged in as: loubnaelmeddah (loubnaelmeddah-oberlin-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
api = wandb.Api()

entity = "loubnaelmeddah"        # replace with your wandb username/team
project = "faithful-puddle-2"   # replace with your project name
     # your run ID from wandb dashboard

run = api.run("loubnaelmeddah-oberlin-college/huggingface/fhb1zxdo")

# List all files to find your latest checkpoint
for file in run.files():
    print(file.name)


artifact/2149913392/wandb_manifest.json
requirements.txt
wandb-metadata.json


In [1]:
import wandb
run = wandb.init()
artifact = run.use_artifact('loubnaelmeddah-oberlin-college/huggingface/run-fhb1zxdo-history:v0', type='wandb-history')
artifact_dir = artifact.download()

wandb: Currently logged in as: loubnaelmeddah (loubnaelmeddah-oberlin-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


In [3]:
import os
print(os.listdir(artifact_dir))


['0000.parquet']


In [8]:
import pandas as pd

df = pd.read_parquet("/content/artifacts/run-fhb1zxdo-history:v0")
print(df.head())


   train/global_step  train/learning_rate  _step     _runtime    _timestamp  \
0               10.0             0.000018    0.0  1221.689940  1.760749e+09   
1               20.0             0.000038    1.0  2392.781583  1.760750e+09   
2               30.0             0.000058    2.0  3461.171601  1.760751e+09   
3               40.0             0.000076    3.0  4503.338608  1.760752e+09   
4               50.0             0.000096    4.0  5576.123182  1.760753e+09   

   train/loss  train/epoch  train/grad_norm  
0      2.8555     0.052083         0.632462  
1      2.7996     0.104167         0.651423  
2      2.5607     0.156250         0.664466  
3      1.9515     0.208333         3.555250  
4      1.1902     0.260417         0.909164  


In [ ]:
from transformers import LlamaForCausalLM

model_name = "baffo32/decapoda-research-llama-7B-hf"
model = LlamaForCausalLM.from_pretrained(model_name)


#model.load_state_dict(torch.load("/content/artifacts/run-fhb1zxdo-history:v0/0000parquet"))



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The following generation flags are not valid and may be ignored: ['pad_token_id']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [ ]:
import pandas as pd

df = pd.read_parquet("/content/artifacts/run-fhb1zxdo-history:v0")
print(df.head())


   train/global_step  train/learning_rate  _step     _runtime    _timestamp  \
0               10.0             0.000018    0.0  1221.689940  1.760749e+09   
1               20.0             0.000038    1.0  2392.781583  1.760750e+09   
2               30.0             0.000058    2.0  3461.171601  1.760751e+09   
3               40.0             0.000076    3.0  4503.338608  1.760752e+09   
4               50.0             0.000096    4.0  5576.123182  1.760753e+09   

   train/loss  train/epoch  train/grad_norm  
0      2.8555     0.052083         0.632462  
1      2.7996     0.104167         0.651423  
2      2.5607     0.156250         0.664466  
3      1.9515     0.208333         3.555250  
4      1.1902     0.260417         0.909164  


In [ ]:
import torch
from transformers import LlamaForCausalLM
from transformers import LlamaTokenizer

import torch
torch.cuda.empty_cache()



base_model = "baffo32/decapoda-research-llama-7B-hf"

model = LlamaForCausalLM.from_pretrained(base_model, load_in_8bit=True, device_map="auto")
tokenizer = LlamaTokenizer.from_pretrained(base_model)

torch.save(model.state_dict(), 'model_weights.pth')

output_dir = "./lora-alpaca"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)




/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
The following generation flags are not valid and may be ignored: ['pad_token_id']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


In [1]:
import torch
from transformers import LlamaForCausalLM
from transformers import LlamaTokenizer

import torch




base_model = "baffo32/decapoda-research-llama-7B-hf"

model = LlamaForCausalLM.from_pretrained(base_model, load_in_8bit=True, device_map="auto")
tokenizer = LlamaTokenizer.from_pretrained(base_model)

torch.save(model.state_dict(), 'model_weights.pth')

output_dir = "./lora-alpaca"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)




/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/428 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ImportError: Using `bitsandbytes` 8-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [ ]:
if epoch % 1 == 0:
    torch.save(model.state_dict(), f"checkpoint_epoch_{epoch}.pt")
    wandb.save(f"checkpoint_epoch_{epoch}.pt")


In [ ]:
artifacts = run.logged_artifacts()
for artifact in artifacts:
    print(f"Artifact name: {artifact.name}, Type: {artifact.type}")


In [ ]:
pip install -r requirements.txt


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
!pip install torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0
!pip install transformers peft bitsandbytes datasets fire pillow==11.3.0


In [ ]:
import torch
import transformers
from transformers import LlamaForCausalLM
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import bitsandbytes as bnb
import datasets
import fire
import PIL  # Pillow for image-related tasks if needed


In [ ]:
!ls -l


total 8
drwxr-xr-x 6 root root 4096 Oct 17 23:42 alpaca-lora
drwxr-xr-x 1 root root 4096 Oct 16 13:41 sample_data


In [ ]:
!pip install -q torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0
!pip install -q transformers peft bitsandbytes datasets fire


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.49.1 requires pillow<12.0,>=8.0, but you have pillow 12.0.0 which is incompatible.


In [ ]:
!pip install pillow==11.3.0



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 49.2 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 12.0.0
    Uninstalling pillow-12.0.0:
      Successfully uninstalled pillow-12.0.0


In [ ]:
pip install torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 --force-reinstall


  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_cupti_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cudnn_cu12-9.10.2.21-py3-none-manylinux_2_27_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_cublas_cu12-12.8.4.1-py3-none-manylinux_2_27_x86_64.whl.me

In [ ]:
import os
os.chdir('/content/alpaca-lora')
print("Current directory:", os.getcwd())


Current directory: /content/alpaca-lora


In [ ]:
%cd /content/alpaca-lora

/content/alpaca-lora


In [ ]:
!python finetune.py --base_model 'baffo32/decapoda-research-llama-7B-hf' --data_path './output_training_advice.jsonl' --val_data_path './output_testing_advice (1).jsonl' --output_dir './lora-alpaca-output' --batch_size 64 --micro_batch_size 4 --num_epochs 4 --learning_rate 1e-4 --cutoff_len 512 --val_set_size 150 --lora_r 8 --lora_alpha 16 --lora_dropout 0.05 --train_on_inputs True --group_by_length


2025-10-18 00:28:20.812240: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760747300.841379   19164 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760747300.851092   19164 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760747300.885943   19164 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760747300.885973   19164 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760747300.885977   19164 computation_placer.cc:177] computation placer alr

In [ ]:
from transformers import LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained("baffo32/decapoda-research-llama-7B-hf")
print("Max token length:", tokenizer.model_max_length)

from transformers import LlamaForCausalLM

model = LlamaForCausalLM.from_pretrained("baffo32/decapoda-research-llama-7B-hf")
print("Model max position embeddings:", model.config.max_position_embeddings)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


...

pytorch_model-00029-of-00033.bin:   0%|          | 0.00/405M [00:00<?, ?B/s]

pytorch_model-00030-of-00033.bin:   0%|          | 0.00/405M [00:00<?, ?B/s]

pytorch_model-00031-of-00033.bin:   0%|          | 0.00/405M [00:00<?, ?B/s]

pytorch_model-00032-of-00033.bin:   0%|          | 0.00/405M [00:00<?, ?B/s]

pytorch_model-00033-of-00033.bin:   0%|          | 0.00/524M [00:00<?, ?B/s]

In [ ]:

input_text = "The driver is hair_and_makeup, looking_road, and using only_right hand(s) on the wheel. Driver is not_talking and/or not applicable and his eyes close. Weather is clear, traffic is medium, and the car is none speed limit on a urban road. Behavior classification detected: drowsiness,high_distraction."

tokens = tokenizer(input_text)
num_tokens = len(tokens['input_ids'])
print(f"Number of tokens: {num_tokens}")

NameError: name 'tokenizer' is not defined

In [ ]:
!python finetune.py --help



2025-10-15 17:31:47.112091: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760549507.132769   65871 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760549507.139020   65871 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760549507.155253   65871 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760549507.155278   65871 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760549507.155282   65871 computation_placer.cc:177] computation placer alr

In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel

base_model_name = "baffo32/decapoda-research-llama-7B-hf"
adapter_path = "./lora-alpaca-output/checkpoint 6-0/adapter_model.safetensors"  # example path

# Load base model and tokenizer
tokenizer = LlamaTokenizer.from_pretrained(base_model_name)
model = LlamaForCausalLM.from_pretrained(base_model_name)

# Load LoRA adapter
model = PeftModel.from_pretrained(model, adapter_path, is_trainable=False)

model.eval()

# Now you can test/generate with the patched model


input_text = "The driver is hair_and_makeup, looking_road, and using only_right hand(s) on the wheel. Driver is not_talking and/or NA and his eyes close.Weather is clear, traffic is medium, and the car is none speed limit on a urban road. Behavior classification detected: drowsiness,high_distraction."
inputs = tokenizer(input_text, return_tensors="pt")

with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], max_length=100)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

In [ ]:
pwd


'/content/alpaca-lora'

In [ ]:
import transformers
print(transformers.__version__)


4.57.1


In [ ]:
!pip list | grep transformers

sentence-transformers                 5.1.1
transformers                          4.57.1


In [ ]:
!pip show bitsandbytes


Name: bitsandbytes
Version: 0.37.2
Summary: 8-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/TimDettmers/bitsandbytes
Author: Tim Dettmers
Author-email: dettmers@cs.washington.edu
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: 
Required-by: 


In [ ]:
!pip install bitsandbytes-cuda117




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 60.6 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade bitsandbytes bitsandbytes-cuda117
!pip install --upgrade torch torchvision torchaudio
!pip install --upgrade transformers
!pip install --upgrade peft
!pip install --upgrade accelerate


  Using cached bitsandbytes-0.48.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.48.1-py3-none-manylinux_2_24_x86_64.whl (60.1 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.37.2
    Uninstalling bitsandbytes-0.37.2:
      Successfully uninstalled bitsandbytes-0.37.2


In [ ]:
import torch

if torch.cuda.is_available():
    print("GPU is available")
    print("GPU device name:", torch.cuda.get_device_name(0))
else:
    print("GPU is NOT available, running on CPU")


GPU is available
GPU device name: Tesla T4


In [ ]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118


In [ ]:
!pip install bitsandbytes==0.37.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 MB 7.5 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.48.1
    Uninstalling bitsandbytes-0.48.1:
      Successfully uninstalled bitsandbytes-0.48.1


In [ ]:
!rm -rf alpaca-lora
!git clone https://github.com/tloen/alpaca-lora.git



Cloning into 'alpaca-lora'...
remote: Enumerating objects: 607, done.
remote: Total 607 (delta 0), reused 0 (delta 0), pack-reused 607 (from 1)
Receiving objects: 100% (607/607), 27.84 MiB | 17.78 MiB/s, done.
Resolving deltas: 100% (358/358), done.


In [ ]:
!pip list | grep peft


peft                                  0.17.2.dev0


In [ ]:
!pip uninstall -y peft
!pip install peft==0.9.0



Found existing installation: peft 0.17.2.dev0
Uninstalling peft-0.17.2.dev0:
  Successfully uninstalled peft-0.17.2.dev0
  Using cached peft-0.9.0-py3-none-any.whl.metadata (13 kB)
Using cached peft-0.9.0-py3-none-any.whl (190 kB)


In [ ]:
!pip install --upgrade transformers bitsandbytes accelerate


In [ ]:
!pip uninstall -y peft
!pip install peft==0.9.0



Found existing installation: peft 0.17.2.dev0
Uninstalling peft-0.17.2.dev0:
  Successfully uninstalled peft-0.17.2.dev0
  Using cached peft-0.9.0-py3-none-any.whl.metadata (13 kB)
Using cached peft-0.9.0-py3-none-any.whl (190 kB)


In [ ]:
!pip install --upgrade transformers bitsandbytes accelerate
